In [ ]:
from vgrid.utils.geometry import fix_h3_antimeridian_cells
import geopandas as gpd
from shapely.geometry import Polygon

# Read the GeoJSON file
gdf = gpd.read_file('antimeridian_crossed.geojson')

# Function to fix geometry coordinates
def fix_geometry(geom):
    if geom.geom_type == 'Polygon':
        # Fix exterior ring
        coords = geom.exterior.coords[:-1]  # Exclude last duplicate point
        hex_boundary = [(lat, lon) for lon, lat in coords]
        fixed_boundary = fix_h3_antimeridian_cells(hex_boundary)
        fixed_exterior = [[lon, lat] for lat, lon in fixed_boundary]
        fixed_exterior.append(fixed_exterior[0])  # Close the polygon
        
        # Fix interior rings (holes) if any
        fixed_interiors = []
        for interior in geom.interiors:
            coords = interior.coords[:-1]
            hex_boundary = [(lat, lon) for lon, lat in coords]
            fixed_boundary = fix_h3_antimeridian_cells(hex_boundary)
            fixed_interior = [[lon, lat] for lat, lon in fixed_boundary]
            fixed_interior.append(fixed_interior[0])
            fixed_interiors.append(fixed_interior)
        
        return Polygon(fixed_exterior, fixed_interiors)
    elif geom.geom_type == 'MultiPolygon':
        # Fix each polygon in the multipolygon
        fixed_polygons = [fix_geometry(poly) for poly in geom.geoms]
        from shapely.geometry import MultiPolygon
        return MultiPolygon(fixed_polygons)
    return geom

# Apply fix to all geometries
gdf['geometry'] = gdf['geometry'].apply(fix_geometry)

# Return the fixed GeoDataFrame
gdf.to_file('shifted_antimeridian.geojson', driver='GeoJSON')


In [2]:
import geopandas as gpd
from shapely import wkt

# Your WKT polygon
polygon_wkt = "POLYGON((170 40, 170 50, -170 50, -170 40, 170 40))"

# Create a GeoSeries
gdf = gpd.GeoSeries([wkt.loads(polygon_wkt)], crs="EPSG:4326")

# Area in degrees² (approx, just multiply lon/lat differences)
area_deg2 = gdf.area[0]
print("Area in degree²:", area_deg2)

# Optional: Project to an equal-area projection to get km²
gdf_proj = gdf.to_crs("EPSG:6933")  # World Cylindrical Equal Area
area_km2 = gdf_proj.area[0] / 1e6   # convert m² to km²
print("Approximate area in km²:", area_km2)

Area in degree²: 3400.0
Approximate area in km²: 29753321.48249786


C:\Users\TECHCRAFT-QDThang\AppData\Local\Temp\ipykernel_3272\2890344483.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  area_deg2 = gdf.area[0]
